In [1]:
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from peft import get_peft_model, LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# Define matching LoRA config manually
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap base model with LoRA
model = get_peft_model(base_model, lora_config)

# Download adapter weights manually (correct ones)
adapter_path = hf_hub_download(
    repo_id="vignesh0007/Anime-Gen-Llama-2-7B",
    filename="adapter_model.safetensors",
    repo_type="model"
)

# Load LoRA weights into the model
state_dict = load_file(adapter_path)
model.load_state_dict(state_dict, strict=False)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("vignesh0007/Anime-Gen-Llama-2-7B")



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [2]:
def build_prompt(premise):
    return f"""Below are examples of anime short stories, each structured into exactly 5 manga panels:

---

Title: The Last Samurai
Characters: Ryu, Hana, Master Kaito
Panel 1: Ryu trains fiercely under Master Kaito's watchful eyes at dawn.
Panel 2: Hana secretly watches Ryu from behind a cherry blossom tree, smiling gently.
Panel 3: Enemy samurai burst into the dojo; Master Kaito steps forward confidently.
Panel 4: Ryu and Hana fight side-by-side against the attackers, swords clashing dramatically.
Panel 5: With enemies defeated, Ryu and Hana exchange quiet glances beneath falling cherry blossoms.

---

Title: The Spirit Painter
Characters: Mei, Old Master Kuro
Panel 1: Mei enters a forgotten temple, sketchbook in hand, led by whispers of a legendary painter.
Panel 2: She finds a crumbling mural with faded colors that seem to shift when she draws near.
Panel 3: As she sketches, a phantom hand begins painting beside her, matching her every stroke.
Panel 4: The mural comes to life, revealing Old Master Kuro, whose spirit had waited for centuries.
Panel 5: The two artists nod in understanding before he fades, leaving Mei’s sketchbook glowing softly.

---

Now, create another short anime story structured into exactly 5 manga panels based on this premise:

**Premise:** {premise}

Title:"""

In [3]:
def clean_story(generated_text: str):
    match = re.search(r"(Panel 5:.*?)(\n|$)", generated_text, flags=re.DOTALL)
    if match:
        return generated_text[:match.end()].strip()
    return generated_text.strip()


In [4]:
def generate_story_from_premise(premise: str):
    prompt = build_prompt(premise)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.8,
            top_p=0.95,
            repetition_penalty=1.1,
            do_sample=True,
            eos_token_id=None
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    story = full_output[len(prompt):].strip()
    story = clean_story(story)
    return story

In [5]:
import torch, re
premise = "Every time a girl plays a certain song, the sky changes color."
story = generate_story_from_premise(premise)
print(story)

Rainbow Daze  
Characters: Rin, Yumi, Miako  
Panel 1: The three friends walk home together after school, chatting idly about their day.  
Panel 2: Rin asks if anyone likes music and Yumi says she does but can't play any instruments.  
Panel 3: Miako recalls how Rin once taught her piano lessons and played beautifully.  
Panel 4: Rin offers to teach both girls "the most beautiful piece I know" as she hums a tune.  
Panel 5: They all stop, spellbound, as the sky turns shimmering rainbow hues above them. package com.azure.resourcemanager.eventgrid.models;
